In [ ]:
#pip install -r requirements.txt

In [ ]:
#pip install deepface

In [ ]:
%cd C:\Users\dell\HCI_Project_Phase_2\yolov7

In [ ]:
#%cd C:\Users\dell\HCI_Project_Phase_2

In [ ]:
#pip install tensorflow==2.10.0

In [ ]:
#pip install tensorflow-gpu==2.10.0


In [ ]:
#pip uninstall tensorflow

In [ ]:
#pip install --upgrade numpy


In [ ]:
#pip install tensorflow==2.10.0

In [ ]:
#pip install cmake

In [ ]:
#pip install dlib-19.22.99-cp310-cp310-win_amd64.whl

In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
print("TensorFlow version:", tf.__version__)
print("TensorFlow-GPU enabled:", tf.test.is_built_with_cuda())

In [ ]:
import sys
print(sys.version)


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # GPU(s) detected
    for gpu in gpus:
        print("GPU:", gpu)
else:
    print("No GPU detected.")


In [ ]:
#%matplotlib inline

In [ ]:
#pip install --upgrade deepface

In [ ]:
import argparse
import time
from torchvision.transforms import functional as F
from pathlib import Path
import cv2
import torch
import numpy as np
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
from numpy import random
import face_recognition
from collections import deque
from imutils.video import VideoStream
import imutils
import time
from gaze_tracking import GazeTracking
import torchvision.transforms as transforms
import random
import face_recognition
import deepface
from deepface import DeepFace
from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, check_imshow, non_max_suppression, apply_classifier, \
    scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized, TracedModel
from PIL import Image
from playsound import playsound
import winsound
from torchvision.models import resnet34

In [ ]:
def letterbox(img, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True, stride=32):
    # Resize and pad image while meeting stride-multiple constraints
    shape = img.shape[:2]  # current shape [height, width]
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    # Scale ratio (new / old)
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    if not scaleup:  # only scale down, do not scale up (for better test mAP)
        r = min(r, 1.0)

    # Compute padding
    ratio = r, r  # width, height ratios
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding
    if auto:  # minimum rectangle
        dw, dh = np.mod(dw, stride), np.mod(dh, stride)  # wh padding
    elif scaleFill:  # stretch
        dw, dh = 0.0, 0.0
        new_unpad = (new_shape[1], new_shape[0])
        ratio = new_shape[1] / shape[1], new_shape[0] / shape[0]  # width, height ratios

    dw /= 2  # divide padding into 2 sides
    dh /= 2

    if shape[::-1] != new_unpad:  # resize
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    return img, ratio, (dw, dh)

In [ ]:
classes_to_filter = ['train'] #You can give list of classes to filter by name ['train','person' ]


opt  = {
    "weights": "weights/yolov7.pt", # Path to weights file default weights are for nano model
    "yaml"   : "data/coco.yaml",
    "img-size": 640, # default image size
    "conf-thres": 0.25, # confidence threshold for inference.
    "iou-thres" : 0.45, # NMS IoU threshold for inference.
    "device" : 'cpu',  # device to run our model i.e. 0 or 0,1,2,3 or cpu
    "classes" : classes_to_filter  # list of classes to filter or None
}

In [ ]:
file_path = "C:\\Users\\moham\\Downloads\\HCI_Project-main\\HCI_Project-main\\Users.txt"  # Replace with the actual file path
# Initialize the emotion analysis
known_face_names = []  # Replace with your known face names
known_face_encodings = []

with open(file_path, "r") as file:
    for line in file:
        data = line.strip().split(",")
        # data will be a list containing the split values
        
        # Access individual values
        image_path = data[0]
        name = data[1]
        code = data[2]
        
        image = face_recognition.load_image_file(image_path)
        face_encoding = face_recognition.face_encodings(image)[0]  # Assuming there's only one face in each image
        known_face_encodings.append(face_encoding)
        known_face_names.append(name)
        # Do something with the values
        print("Image Path:", image_path)
        print("Name:", name)
        print("Code:", code)
        
        # Additional processing...


In [ ]:
file_path = file_path  # Replace with the actual file path
data_list = []

with open(file_path, "r") as file:
    for line in file:
        data = line.strip().split(",")
        data_list.append(data)

# Print the list
for row in data_list:
    print(row)


In [ ]:
# # Define the lower and upper boundaries of the colors in the HSV color space
colors = {
    'blue': {
        'lower': np.array([90, 100, 100]),
        'upper': np.array([130, 255, 255])
    },
    'yellow': {
        'lower': np.array([20, 100, 100]),
        'upper': np.array([30, 255, 255])
    }
}

# Color labels for drawing text on the frame
color_labels = {
    'blue': (255, 0, 0),
    'yellow': (0, 255, 255)
}

# Initialize the list of tracked points, the frame counter, and the coordinate deltas
pts = deque(maxlen=32)
counter = 0
(dX, dY) = (0, 0)
direction = ""

# Load the face cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load object detection model
weights = 'weights/yolov7.pt'  # Replace with the path to your YOLOv7 weights
imgsz = 416  # Set the input image size for the model

# Set device and half precision
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
half = device.type != 'cpu'

# Load model
model = attempt_load(weights, map_location=device)
stride = int(model.stride.max())
imgsz = check_img_size(imgsz, s=stride)  # Check image size

if half:
    model.half()  # To FP16

names = model.module.names if hasattr(model, 'module') else model.names
color_list = [[random.randint(0, 255) for _ in range(3)] for _ in names]

gaze = GazeTracking()
# Initialize a timer and a variable to track the duration of closed eyes
closed_eye_timer = None
closed_eye_duration = 0
sleep_threshold = 7  # Number of seconds to consider as sleeping
video_capture = cv2.VideoCapture(0)  # 0 represents the default camera
# cv2.namedWindow("Real-time Emotion Analysis")

# Allow the camera to warm up
time.sleep(2.0)

# Calculate the distance between two points
def calculate_distance(point1, point2):
    return np.sqrt((point2[0] - point1[0]) ** 2 + (point2[1] - point1[1]) ** 2)

while True:
    # Read the next frame from the video stream
    ret, frame = video_capture.read()
    if not ret:
        break

    # Face detection
    # Load face recognition model
    for data in data_list:
        image_path = data[0]
        name = data[1]
        Password = data[2]
    
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        rgb_small_frame = small_frame[:, :, ::-1]
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
    
    face_names = []
    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"

        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_face_names[best_match_index]

        face_names.append(name)

        # Display the Face Detection results
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
    
    # Gaze Tracking
    gaze.refresh(frame)
    new_frame = gaze.annotated_frame()
    text = ""
    if gaze.is_right():
        text = "Looking right"
    elif gaze.is_left():
        text = "Looking left"
    elif gaze.is_center():
        text = "Looking center"
    elif gaze.is_blinking():
        if closed_eye_timer is None:
            # Start the timer if the eyes are closed for the first time
            closed_eye_timer = time.time()
        else:
            # Calculate the duration of closed eyes
            closed_eye_duration = time.time() - closed_eye_timer

            if closed_eye_duration >= sleep_threshold:
                text = "He is sleeping"
            else:
                text = "Blinking"
    else:
        # Reset the timer if the eyes are open
        closed_eye_timer = None
        closed_eye_duration = 0

    cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Resize the frame, blur it, and convert it to the HSV color space
    frame = imutils.resize(frame, width=600)
    blurred = cv2.GaussianBlur(frame, (11, 11), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

    # Detect objects using YOLOv7
    img = letterbox(frame, imgsz, stride=stride)[0]
    img = img.transpose((2, 0, 1))[::-1]
    img = np.ascontiguousarray(img)

    img = torch.from_numpy(img).to(device)
    img = img.half() if half else img.float()
    img /= 255.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # Inference
    t1 = time_synchronized()
    pred = model(img, augment=False)[0]
    pred = non_max_suppression(pred, 0.4, 0.5, classes=None, agnostic=False)
    t2 = time_synchronized()

    # Process detections
    for i, det in enumerate(pred):  # Detections per image
        if len(det):
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], frame.shape).round()

            # Iterate over detections and draw bounding boxes
            for *xyxy, conf, cls in reversed(det):
                c = int(cls)  # Integer class
                label = f'{names[c]} {conf:.2f}'
                color = color_list[c]

                # Draw bounding boxes
                cv2.rectangle(frame, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), color, 2)
                cv2.putText(frame, label, (int(xyxy[0]), int(xyxy[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

                # Convert the frame to grayscale and equalize the histogram
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                gray = cv2.equalizeHist(gray)

                # Detect faces in the grayscale frame
                faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30),
                                                      flags=cv2.CASCADE_SCALE_IMAGE)

                # Iterate over the detected faces
                for (x, y, w, h) in faces:
                    # Compute the center of the face
                    center = (x + w // 2, y + h // 2)

                    # Update the points queue
                    pts.appendleft(center)

                    # Draw a rectangle around the face
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

                    # Draw the center of the face
                    cv2.circle(frame, center, 5, (0, 0, 255), -1)

                    # Calculate the distance between the center of the face and the color
                    color_name = names[c]
                    color_center = (int((xyxy[0] + xyxy[2]) / 2), int((xyxy[1] + xyxy[3]) / 2))
                    for color, bounds in colors.items():
                        lower = bounds['lower']
                        upper = bounds['upper']
                        if lower[0] <= hsv[color_center[1], color_center[0], 0] <= upper[0]:
                            distance = calculate_distance(center, color_center)

                            # Draw a line between the face and the color
                            cv2.line(frame, center, color_center, color_labels[color], 2)

                            # Check if the distance is less than 100
                            if distance < 100:
                                # Play a sound
                                winsound.Beep(1000, 500)  # Beep with frequency 1000Hz for 500 milliseconds

                            # Label the color in the stream
                            cv2.putText(frame, color_name, (int(xyxy[0]), int(xyxy[1]) + 40), cv2.FONT_HERSHEY_SIMPLEX,
                                        0.9, color_labels[color], 2)

                    # Detect face expression
                    # Perform emotion analysis on the frame
                    result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)

                    # Retrieve the dominant emotion from the first result in the list
                    dominant_emotion = result[0]['dominant_emotion']

                    # Display the dominant emotion label on the frame
                    cv2.putText(
                        frame,
                        dominant_emotion,
                        (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.9,
                        (0, 255, 0),
                        2
                    )

    # Show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # If the 'q' key is pressed, break from the loop
    if key == ord("q"):
        break

# Release the video stream and close any open windows
video_capture.release()
cv2.destroyAllWindows()


In [ ]:
# # Define the lower and upper boundaries of the colors in the HSV color space
# colors = {
#    'blue': {
#         'lower': np.array([90, 100, 100]),
#         'upper': np.array([130, 255, 255])
#     },
#     'yellow': {
#         'lower': np.array([20, 100, 100]),
#         'upper': np.array([30, 255, 255])
#     }
# }

# # Color labels for drawing text on the frame
# color_labels = {
#     'blue': (255, 0, 0),
#     'yellow': (0, 255, 255)
# }

# # Initialize the list of tracked points, the frame counter, and the coordinate deltas
# pts = deque(maxlen=32)
# counter = 0
# (dX, dY) = (0, 0)
# direction = ""
# # Load the face cascade classifier
# face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# # Load object detection model
# weights = 'weights/yolov7.pt'  # Replace with the path to your YOLOv7 weights
# imgsz = 416  # Set the input image size for the model

# # Set device and half precision
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# half = device.type != 'cpu'

# # Load model
# model = attempt_load(weights, map_location=device)
# stride = int(model.stride.max())
# imgsz = check_img_size(imgsz, s=stride)  # Check image size

# if half:
#     model.half()  # To FP16

# names = model.module.names if hasattr(model, 'module') else model.names
# color_list = [[random.randint(0, 255) for _ in range(3)] for _ in names]

# # Load face recognition model
# Supra_image = face_recognition.load_image_file("Supraa.jpg")
# Supra_face_encoding = face_recognition.face_encodings(Supra_image)[0]
# known_face_encodings = [Supra_face_encoding]  # Replace with your known face encodings
# known_face_names = ['Supraa','Mortaga','Menna']  # Replace with your known face names

# # Initialize the emotion analysis
# gaze = GazeTracking()
# video_capture = cv2.VideoCapture(0)  # 0 represents the default camera
# #cv2.namedWindow("Real-time Emotion Analysis")

# # Allow the camera to warm up
# time.sleep(2.0)
# # Calculate the distance between two points
# def calculate_distance(point1, point2):
#     return np.sqrt((point2[0] - point1[0]) ** 2 + (point2[1] - point1[1]) ** 2)

# while True:
#     # Read the next frame from the video stream
#     ret, frame = video_capture.read()
#     if not ret:
#         break

#     # Face detection
#     small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
#     rgb_small_frame = small_frame[:, :, ::-1]
#     face_locations = face_recognition.face_locations(rgb_small_frame)
#     face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
#     face_names = []

#     for face_encoding in face_encodings:
#         matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
#         name = "Unknown"

#         face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
#         best_match_index = np.argmin(face_distances)
#         if matches[best_match_index]:
#             name = known_face_names[best_match_index]

#         face_names.append(name)

#     # Display the Face Detection results
#     for (top, right, bottom, left), name in zip(face_locations, face_names):
#         top *= 4
#         right *= 4
#         bottom *= 4
#         left *= 4

#         cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
#         cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
#         font = cv2.FONT_HERSHEY_DUPLEX
#         cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

#     gaze.refresh(frame)
#     new_frame = gaze.annotated_frame()
#     text = ""
#     if gaze.is_right():
#         text = "Looking right"
#     elif gaze.is_left():
#         text = "Looking left"
#     elif gaze.is_center():
#         text = "Looking center"
#     elif gaze.is_blinking():
#         text = "Blinking"

#     cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

#     # Resize the frame, blur it, and convert it to the HSV color space
#     frame = imutils.resize(frame, width=600)
#     blurred = cv2.GaussianBlur(frame, (11, 11), 0)
#     hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

#     # Detect objects using YOLOv7
#     img = letterbox(frame, imgsz, stride=stride)[0]
#     img = img.transpose((2, 0, 1))[::-1]
#     img = np.ascontiguousarray(img)

#     img = torch.from_numpy(img).to(device)
#     img = img.half() if half else img.float()
#     img /= 255.0
#     if img.ndimension() == 3:
#         img = img.unsqueeze(0)

#     # Inference
#     t1 = time_synchronized()
#     pred = model(img, augment=False)[0]
#     pred = non_max_suppression(pred, 0.4, 0.5, classes=None, agnostic=False)
#     t2 = time_synchronized()

#     # Process detections
#     for i, det in enumerate(pred):  # Detections per image
#         if len(det):
#             det[:, :4] = scale_coords(img.shape[2:], det[:, :4], frame.shape).round()

#             # Iterate over detections and draw bounding boxes
#             for *xyxy, conf, cls in reversed(det):
#                 c = int(cls)  # Integer class
#                 label = f'{names[c]} {conf:.2f}'
#                 color = color_list[c]

#                 # Draw bounding boxes
#                 cv2.rectangle(frame, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), color, 2)
#                 cv2.putText(frame, label, (int(xyxy[0]), int(xyxy[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

#                 # Convert the frame to grayscale and equalize the histogram
#                 gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#                 gray = cv2.equalizeHist(gray)

#                 # Detect faces in the grayscale frame
#                 faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30),
#                                                       flags=cv2.CASCADE_SCALE_IMAGE)

#                 # Iterate over the detected faces
#                 for (x, y, w, h) in faces:
#                     # Compute the center of the face
#                     center = (x + w // 2, y + h // 2)

#                     # Update the points queue
#                     pts.appendleft(center)

#                     # Draw a rectangle around the face
#                     cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

#                     # Draw the center of the face
#                     cv2.circle(frame, center, 5, (0, 0, 255), -1)

#                     # Calculate the distance between the center of the face and the color
#                     color_name = names[c]
#                     color_center = (int((xyxy[0] + xyxy[2]) / 2), int((xyxy[1] + xyxy[3]) / 2))
#                     for color, bounds in colors.items():
#                         lower = bounds['lower']
#                         upper = bounds['upper']
#                         if lower[0] <= hsv[color_center[1], color_center[0], 0] <= upper[0]:
#                             distance = calculate_distance(center, color_center)

#                             # Draw a line between the face and the color
#                             cv2.line(frame, center, color_center, color_labels[color], 2)

#                             # Check if the distance is less than 100
#                             if distance < 100:
#                                 # Play a sound
#                                 winsound.Beep(1000, 500)  # Beep with frequency 1000Hz for 500 milliseconds

#                             # Label the color in the stream
#                             cv2.putText(frame, color_name, (int(xyxy[0]), int(xyxy[1]) + 40), cv2.FONT_HERSHEY_SIMPLEX,
#                                         0.9, color_labels[color], 2)

#                     # Detect face expression
#                     # Perform emotion analysis on the frame
#                     result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)

#                     # Retrieve the dominant emotion from the first result in the list
#                     dominant_emotion = result[0]['dominant_emotion']

#                     # Display the dominant emotion label on the frame
#                     cv2.putText(
#                         frame,
#                         dominant_emotion,
#                         (x, y - 10),
#                         cv2.FONT_HERSHEY_SIMPLEX,
#                         0.9,
#                         (0, 255, 0),
#                         2
#                     )

#     # Show the output frame
#     cv2.imshow("Frame", frame)
#     key = cv2.waitKey(1) & 0xFF

#     # If the 'q' key is pressed, break from the loop
#     if key == ord("q"):
#         break

# # Release the video stream and close any open windows
# video_capture.release()
# cv2.destroyAllWindows()


In [ ]:

# # Video information
# fps = 30  # Adjust the frame rate as needed
# w, h = 640, 480  # Set the desired width and height for the video output

# # Initializing video object
# video = cv2.VideoCapture(0)  # Use index 0 for the default camera

# # Initializing object for writing video output
# #output = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'DIVX'), fps, (w, h))

# # Initializing model and setting it for inference
# with torch.no_grad():
#     weights = 'weights/yolov7.pt'  # Replace with the path to your YOLOv7 weights
#     imgsz = 416  # Set the input image size for the model

#     set_logging()
#     device = select_device('')
#     half = device.type != 'cpu'
#     model = attempt_load(weights, map_location=device)
#     stride = int(model.stride.max())
#     imgsz = check_img_size(imgsz, s=stride)
#     if half:
#         model.half()

#     names = model.module.names if hasattr(model, 'module') else model.names
#     colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]
#     if device.type != 'cpu':
#         model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))

#     while True:
#         ret, img0 = video.read()

#         if ret:
#             img = letterbox(img0, imgsz, stride=stride)[0]
#             img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
#             img = np.ascontiguousarray(img)
#             img = torch.from_numpy(img).to(device)
#             img = img.half() if half else img.float()
#             img /= 255.0
#             if img.ndimension() == 3:
#                 img = img.unsqueeze(0)

#             # Inference
#             t1 = time_synchronized()
#             pred = model(img, augment=False)[0]
#             pred = non_max_suppression(pred, conf_thres=0.3, iou_thres=0.6)

#             t2 = time_synchronized()
#             for i, det in enumerate(pred):
#                 if len(det):
#                     det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()

#                     for *xyxy, conf, cls in reversed(det):
#                         label = f'{names[int(cls)]} {conf:.2f}'
#                         plot_one_box(xyxy, img0, label=label, color=colors[int(cls)], line_thickness=3)

#             cv2.imshow('Object Detection', img0)
#             #output.write(cv2.resize(img0, (w, h)))

#             if cv2.waitKey(1) == ord('q'):  # Press 'q' to exit
#                 break
#         else:
#             break

# #output.release()
# video.release()
# cv2.destroyAllWindows()


In [ ]:

# # Initialize video capture object
# gaze = GazeTracking()
# video_capture = cv2.VideoCapture(0)  # Use index 0 for the default camera
# # Load a sample picture and learn how to recognize it.
# Supra_image = face_recognition.load_image_file("Supraa.jpg")
# Supra_face_encoding = face_recognition.face_encodings(Supra_image)[0]
# # Load face recognition model and known face encodings
# known_face_encodings = [Supra_face_encoding]  # Replace with your known face encodings
# known_face_names = ['Supraa']  # Replace with your known face names

# # Load object detection model
# weights = 'weights/yolov7.pt'  # Replace with the path to your YOLOv7 weights
# imgsz = 416  # Set the input image size for the model

# set_logging()
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# half = device.type != 'cpu'
# model = attempt_load(weights, map_location=device)
# stride = int(model.stride.max())
# imgsz = check_img_size(imgsz, s=stride)
# if half:
#     model.half()

# names = model.module.names if hasattr(model, 'module') else model.names
# colors = [[np.random.randint(0, 255) for _ in range(3)] for _ in names]
# if device.type != 'cpu':
#     model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))

# # Process frames from the video capture
# while True:
#     # Grab a single frame of video
#     ret, frame = video_capture.read()

#     if ret:
#         # Face detection
#         small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
#         rgb_small_frame = small_frame[:, :, ::-1]
#         face_locations = face_recognition.face_locations(rgb_small_frame)
#         face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

#         gaze.refresh(frame)

#         new_frame = gaze.annotated_frame()
#         text = ""
#         if gaze.is_right():
#             text = "Looking right"
#         elif gaze.is_left():
#             text = "Looking left"
#         elif gaze.is_center():
#             text = "Looking center"
#         elif gaze.is_blinking():
#             text = "Blinking"
            
#         cv2.putText(new_frame, text, (60, 60), cv2.FONT_HERSHEY_DUPLEX, 2, (255, 0, 0), 2)
#         print(text)
#         face_names = []
#         for face_encoding in face_encodings:
#             matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
#             name = "Unknown"

#             face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
#             best_match_index = np.argmin(face_distances)
#             if matches[best_match_index]:
#                 name = known_face_names[best_match_index]

#             face_names.append(name)

#         # Object detection
#         img0 = frame
#         img = letterbox(img0, imgsz, stride=stride)[0]
#         img = img[:, :, ::-1].transpose(2, 0, 1)
#         img = np.ascontiguousarray(img)
#         img = torch.from_numpy(img).to(device)
#         img = img.half() if half else img.float()
#         img /= 255.0
#         if img.ndimension() == 3:
#             img = img.unsqueeze(0)

#         # Inference
#         t1 = time_synchronized()
#         pred = model(img, augment=False)[0]
#         pred = non_max_suppression(pred, conf_thres=0.3, iou_thres=0.6)
#         t2 = time_synchronized()

#         # Display the results
#         for (top, right, bottom, left), name in zip(face_locations, face_names):
#             top *= 4
#             right *= 4
#             bottom *= 4
#             left *= 4

#             cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
#             cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
#             font = cv2.FONT_HERSHEY_DUPLEX
#             cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

#         for i, det in enumerate(pred):
#             if len(det):
#                 det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()

#                 for *xyxy, conf, cls in reversed(det):
#                     label = f'{names[int(cls)]} {conf:.2f}'
#                     plot_one_box(xyxy, img0, label=label, color=colors[int(cls)], line_thickness=3)

#         cv2.imshow('Video', frame)
#         if cv2.waitKey(1) == ord('q'):  # Press 'q' to exit
#             break
#     else:
#         break

# # Release resources
# video_capture.release()
# cv2.destroyAllWindows()
